In [2]:
cd "C:\Users\deshp\darkflow"

C:\Users\deshp\darkflow


In [1]:
from platform import python_version

print(python_version())

3.7.10


In [2]:
import cv2

In [6]:
#pip install numpy==1.20.0

In [7]:
#!pip install tensorflow==1.13.2 

In [3]:
import warnings
warnings.filterwarnings('ignore')
from darkflow.net.build import TFNet # Importing and buolding the yolo model on predefined yolo weights
import numpy as np
import cv2
import tensorflow as tf

# Config TF, set True if using GPU
config = tf.ConfigProto(log_device_placement = False)
config.gpu_options.allow_growth = False 

with tf.Session(config=config) as sess:
    options = {
            'model': './cfg/yolo.cfg',
            'load': './bin/yolo.weights',
            'threshold': 0.3,
            'gpu': 1.0 # uncomment these if using GPU
               }
    tfnet = TFNet(options)    

Parsing ./cfg/yolo.cfg
Loading ./bin/yolo.weights ...
Successfully identified 203934260 bytes
Finished in 0.035068511962890625s
Model has a coco model name, loading coco labels.

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
Instructions for updating:
Colocations handled automatically by placer.
       |        | input                            | (?, 608, 608, 3)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 608, 608, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 304, 304, 32)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 304, 304, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | maxp 2x2p0_2

In [8]:
#pip install h5py==2.9

In [9]:
#pip install h5py==2.9 --user

In [10]:
#pip install opencv-contrib-python

In [ ]:
from scipy.spatial import distance as dist
capture=cv2.VideoCapture(r"C:\Users\deshp\Downloads\AVSS_E2_001_001_001_001_001_001 (online-video-cutter.com).mp4")
def Display_Results(results,img): 
        for (i,result) in enumerate(results):
            x=result['topleft']['x']
            w=result['bottomright']['x']-result['topleft']['x']
            y=result['topleft']['y']
            h=result['bottomright']['y']-result['topleft']['y']
            cv2.rectangle(img,(x,y),(x+w,y+h),(0.255,0),2)
            label_position = (x+int(w/2),abs(y-10))
            cv2.putText(img,'Unattended Baggage Detected',label_position,cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
        return img
frame_no=0
while True:
    frame_no+=1
    no_of_people=0
    ret , frame = capture.read()
    if ret:
        #print(frame_no)
        results=tfnet.return_predict(frame)
        counter_person=0
        counter_suitcase=0
        counter_handbag=0
        people_list=[]
        suitcase_list=[]
        handbag_list=[]
        people={}
        suitcase={}
        handbag={}
        for (i,result) in enumerate(results):
            if result['label']=='person':
                counter_person+=1
                people[f'person_{counter_person}']=result
                a=(result['bottomright']['x'])/2
                b=(result['topleft']['y'])/2
                aa=tuple((a,b))
                person_centroid=aa
                people[f'person_{counter_person}']['centroid']=person_centroid
                people_list.append(people)
            if result['label']=='suitcase':
                result_suit=result
                counter_suitcase+=1
                suitcase[f'suitcase{counter_suitcase}']=result
                a=(result['bottomright']['x'])/2
                b=(result['topleft']['y'])/2
                aa=tuple((a,b))
                suitcase_centroid=aa
                suitcase[f'suitcase{counter_suitcase}']['centroid']=suitcase_centroid
                suitcase_list.append(suitcase)
            if result['label']=='handbag':
                counter_handbag+=1
                handbag[f'handbag{counter_handbag}']=result
                a=(result['bottomright']['x'])/2
                b=(result['topleft']['y'])/2
                aa=tuple((a,b))
                handbag_centroid=aa
                handbag[f'handbag{counter_handbag}']['centroid']=handbag_centroid
                handbag_list.append(handbag)
            final_distance_handbag=[]
            counter_handbag=0
            for handbags in handbag_list:
                counter_handbag+=1
                inter_distance=[]
                counter_person=0
                for person in people_list:
                    counter_person+=1
                    a = dist.euclidean(person[f'person_{counter_person}']['centroid'], handbags[f'handbag{counter_handbag}']['centroid'])
                    inter_distance.append(a)
                final_distance_handbag.append(inter_distance)
            print(f'final_distances handbag = {final_distance_handbag}')
            final_distance_suitcase=[]
            counter_suitcase=0
            for suitcases in suitcase_list:
                counter_suitcase+=1
                inter_distance=[]
                counter_person=0
                for person in people_list:
                    counter_person+=1
                    a = dist.euclidean(person[f'person_{counter_person}']['centroid'], suitcases[f'suitcase{counter_suitcase}']['centroid'])
                    inter_distance.append(a)
                final_distance_suitcase.append(inter_distance)
            print(f'final_distances suitcase = {final_distance_suitcase}')
            try: 
                final_distance_suitcase[0].sort()
                if final_distance_suitcase[0][0]>125:
                    x=result_suit['topleft']['x']
                    w=result_suit['bottomright']['x']-result_suit['topleft']['x']
                    y=result_suit['topleft']['y']
                    h=result_suit['bottomright']['y']-result_suit['topleft']['y']
                    cv2.rectangle(frame,(x,y),(x+w,y+h),(0.255,0),2)
                    label_position = (x+int(w/2),abs(y-10))
                    cv2.putText(frame,'Unattended Baggage Detected',label_position,cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
            except:
                pass
        cv2.imshow('Yolo Object Detection',frame)
        
        if cv2.waitKey(1) == 13:
            break
capture.release()
cv2.destroyAllWindows()

final_distances handbag = []
final_distances suitcase = []
final_distances handbag = []
final_distances suitcase = []
final_distances handbag = []
final_distances suitcase = [[138.9685216155083, 67.24024092758741]]
final_distances handbag = []
final_distances suitcase = []
final_distances handbag = []
final_distances suitcase = []
final_distances handbag = []
final_distances suitcase = [[139.32874793092773, 67.24024092758741]]
final_distances handbag = []
final_distances suitcase = []
final_distances handbag = []
final_distances suitcase = []
final_distances handbag = []
final_distances suitcase = [[139.67551682381563, 66.30422309325402]]
final_distances handbag = []
final_distances suitcase = []
final_distances handbag = []
final_distances suitcase = []
final_distances handbag = []
final_distances suitcase = [[139.67551682381563, 67.4166151627327]]
final_distances handbag = []
final_distances suitcase = []
final_distances handbag = []
final_distances suitcase = []
final_distances hand